# Solving Complex Tasks with Nested Chats

This notebook shows how you can leverage **nested chats** to solve complex task with AutoGen. Nested chats is a sequence of chats created by a receiver agent after receiving a message from a sender agent and finished before the receiver agent replies to this message. Nested chats allow AutoGen agents to use other agents as their inner monologue to accomplish tasks. This abstraction is powerful as it allows you to compose agents in rich ways. This notebook shows two basic examples of nested chat.

\:\:\:info Requirements

Install `pyautogen`:
```bash
pip install pyautogen
```

For more information, please refer to the [installation guide](/docs/installation/).

\:\:\:

In [1]:
%cd ..

/workspaces/cv-optimizer


/usr/local/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
from utils.filestore import get_completed_cv_data,get_cv_blueprint,get_position_data,set_position_cv_offers
import json
position_data = get_position_data()
cv_data = get_completed_cv_data()
cv_blueprint = get_cv_blueprint()

In [3]:
from typing_extensions import Annotated
import os
from dotenv import load_dotenv

load_dotenv()

import autogen

config_list = [{"model": "gpt-3.5-turbo", "api_key":os.environ['OPENAI_API_KEY'] , "cache_seed": 42}]

## Scenario 0

In [4]:
# hr = autogen.AssistantAgent(
#     llm_config={"config_list": config_list},
#     name="External_recruiter",
#     description="external HR recruit that will help me get the position,",
#     system_message=f"""
#     You are an independent HR recruiter, committed to referring the perfect candidate for the job. 
#     You help candidates to optimize the CV for the position, optimize the CV and output it in the following format:
#     ```json
#     {json.dumps(cv_blueprint,indent=4)}
#     ```
#     You must not fabricate information! 
#     If you find some requirements in the position that you assume will prevent the user CV's to be accepted for an interview, you can ask the user about their experience only, you can use ask_human_expert function.
#     """,
# )

# hiring = autogen.AssistantAgent(
#     name= "Hiring_technical_recruiter",
#     description="technical recruiter in the hiring company",
#     system_message=f"""
#         You are an experienced technical recruiter tasked with filling an open position in a leading tech company. 
#         Your goal is to find the best candidate who not only possesses the necessary technical skills but 
#         also fits well with the company culture.  Your Job is to give critical feedback on the CV you receive to the position.
#         Be concise, professional, and engaging in your communication.

#         The position you are hiring is:
#         {json.dumps(position_data,indent=4)}
#         """,
#     llm_config={"config_list": config_list}
    
# )

# @hr.register_for_llm(description="Function for asking user for more details about it's experience")
# def ask_human_expert(question: Annotated[str, "The question you want to ask the user about is experience."]) -> Annotated[str, "Answer"]:
#   answer = input(f"Please answer the question: {question}\n")
#   return answer

# user_proxy.initiate_chat(assistant, "Hey what is the age of the bedrock in grand canyon?")


### Example Task

Suppose we want the agents to complete the following sequence of tasks:

In [5]:
task = f"""
Hey there,
I've come across this amazing job opportunity that I'm excited about, and I want to make sure my CV is perfectly tailored to it. I've attached the job description below so you can get a sense of what they're looking for.

Could you please review my CV and make any necessary adjustments to better align it with the job description? I want to make sure I highlight the relevant skills and experiences without making it obvious that I've optimized it. 
Also, please make sure not to add any information that isn't already in my CV.

Thanks so much for your help, I really appreciate it!
My CV:
{json.dumps(cv_data,indent=4)}
                        
Position Description:
{json.dumps(position_data,indent=4)}
"""

## Scenario 1
    
Let's say we desire the following workflow to solve the task: a user_proxy agent issues the initial query to a writer and acts as a proxy for the user. Whenever an initial writing is provided, a critic should be invoked to offer critique as feedback. This workflow can be realized by a three-agent system shown below. The system includes a user_proxy agent and a writer agent communicating with each other, with a critic agent nested within the user_proxy agent to provide critique. Whenever the user_proxy receives a message from the writer, it engages in a conversation with the critic agent to work out feedback on the writer's message.

![](nested_chat_1.png)

### Step 1. Define Agents
Define the agents, including the outer agents writer and user_proxy, and the inner agent critic.

In [6]:
writer = autogen.AssistantAgent(
    llm_config={"config_list": config_list},
    name="External_recruiter",
    description="external HR recruit that will help me get the position,",
    system_message=f"""
    You are an independent HR recruiter, committed to referring the perfect candidate for the job. 
    You help candidates to optimize the CV for the position, optimize the CV and output it in the following format:
    ```json
    {json.dumps(cv_blueprint,indent=4)}
    ```
    You must not fabricate information! 
    If you find some requirements in the position that you assume will prevent the user CV's to be accepted for an interview, you can ask the user about their experience only, you can use ask_human_expert function.
    """,
)

@writer.register_for_llm(description="Function for asking user for more details about it's experience")
def ask_human_expert(question: Annotated[str, "The question you want to ask the user about is experience."]) -> Annotated[str, "Answer"]:
  answer = input(f"Please answer the question: {question}\n")
  return answer

def is_termination_msg(x):
    result = x.get("content", "")
    if result is None:
        return False
    return result.find("TERMINATE") >= 0,

user_proxy = autogen.UserProxyAgent(
    llm_config={"config_list": config_list},
    human_input_mode="NEVER",
    is_termination_msg=is_termination_msg,
    name="User_proxy",
    description="The User that would like to submit its CV",
    system_message=f"""You are looking to get an interview, your CV is:
        {json.dumps(cv_data,indent=4)}
        """
)

critic = autogen.AssistantAgent(
    name= "Hiring_technical_recruiter",
    description="technical recruiter in the hiring company",
    system_message=f"""
        You are an experienced technical recruiter tasked with filling an open position in a leading tech company. 
        Your goal is to find the best candidate who not only possesses the necessary technical skills but 
        also fits well with the company culture.  Your Job is to give critical feedback on the CV you receive to the position.
        Be concise, professional, and engaging in your communication.

        The position you are hiring is:
        {json.dumps(position_data,indent=4)}
        """,
    llm_config={"config_list": config_list}
    
)



### Step 2: Orchestrate Nested Chats to Solve Tasks


In [7]:

def reflection_message(recipient, messages, sender, config):
    print("Reflecting...", "yellow")
    return f"While you hire me? {recipient.chat_messages_for_summary(sender)[-1]['content']}"


user_proxy.register_nested_chats(
    [{"recipient": critic, "message": reflection_message, "summary_method": "last_msg", "max_turns": 1}],
    trigger=writer,  # condition=my_condition,
)

res = user_proxy.initiate_chat(recipient=writer, message=task, max_turns=30, summary_method="last_msg")

User_proxy (to External_recruiter):


Hey there,
I've come across this amazing job opportunity that I'm excited about, and I want to make sure my CV is perfectly tailored to it. I've attached the job description below so you can get a sense of what they're looking for.

Could you please review my CV and make any necessary adjustments to better align it with the job description? I want to make sure I highlight the relevant skills and experiences without making it obvious that I've optimized it. 
Also, please make sure not to add any information that isn't already in my CV.

Thanks so much for your help, I really appreciate it!
My CV:
{
    "personal_info": {
        "firstname": "Sefi",
        "lastname": "Erlich",
        "email": "Berlichsefi@gmail.com",
        "phone": "+972 524 307 093",
        "address": "TLV, Israel - willing to relocate",
        "linkedin": {
            "link": "https://www.linkedin.com/in/erlichsefi/",
            "username": []
        },
        "github":

BadRequestError: Error code: 400 - {'error': {'message': "An assistant message with 'tool_calls' must be followed by tool messages responding to each 'tool_call_id'. The following tool_call_ids did not have response messages: call_Xk7PfCZgQOu0isB66fSKKZdE, call_6rtQHotUkNYJNgU491bVGmBz, call_c1OOhZ92x7j4MhX0H5YeopHR, call_M16JPdMEZsfYLvp7ZTKB6jaC", 'type': 'invalid_request_error', 'param': 'messages.[3].role', 'code': None}}

## Scenarios 2
Let's say we desire the following workflow to solve the task. Compared to scenario 1, we want to include an additional `critic_executor` agent to chat with the `critic` and execute some tool calls involved in the chat. For example, a tool for detecting harmful content in the output of the writer.

This workflow can be realized by a four-agent system shown below. The system includes a user_proxy agent and a writer agent communicating with each other, with a chat between the `critic` and `critic_executor` agent nested within the `user_proxy` agent to provide critique. Whenever the user_proxy receives a message from the writer, it engages in a conversation between `critic` and `critic_executor` to work out feedback on the writer's message. A summary of the nested conversation will be passed to the user_proxy, which will then be passed to the writer as feedback.

![](nested_chat_2.png)


In [10]:
critic_executor = autogen.UserProxyAgent(
    name="Critic_Executor",
    human_input_mode="NEVER",
    # is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    code_execution_config={
        "last_n_messages": 1,
        "work_dir": "tasks",
        "use_docker": False,
    },  # Please set use_docker=True if docker is available to run the generated code. Using docker is safer than running the generated code directly.
)

# one way of registering functions is to use the register_for_llm and register_for_execution decorators


@critic_executor.register_for_execution()
@critic.register_for_llm(name="check_harmful_content", description="Check if content contain harmful keywords.")
def check_harmful_content(content: Annotated[str, "Content to check if harmful keywords."]):
    # List of harmful keywords for demonstration purposes
    harmful_keywords = ["violence", "hate", "bullying", "death"]

    # Normalize the input text to lower case to ensure case-insensitive matching
    text = content.lower()

    print(f"Checking for harmful content...{text}", "yellow")
    # Check if any of the harmful keywords appear in the text
    for keyword in harmful_keywords:
        if keyword in text:
            return "Denied. Harmful content detected:" + keyword  # Harmful content detected

    return "Approve. TERMINATE"  # No harmful content detected


def reflection_message_no_harm(recipient, messages, sender, config):
    print("Reflecting...", "yellow")
    return f"Reflect and provide critique on the following writing. Ensure it does not contain harmful content. You can use tools to check it. \n\n {recipient.chat_messages_for_summary(sender)[-1]['content']}"


user_proxy.register_nested_chats(
    [
        {
            "sender": critic_executor,
            "recipient": critic,
            "message": reflection_message_no_harm,
            "max_turns": 2,
            "summary_method": "last_msg",
        }
    ],
    trigger=writer,  # condition=my_condition,
)

res = user_proxy.initiate_chat(recipient=writer, message=task, max_turns=2, summary_method="last_msg")

The return type of the function 'check_harmful_content' is not annotated. Although annotating it is optional, the function should return either a string, a subclass of 'pydantic.BaseModel'.


User (to Writer):

Write a concise but engaging blogpost about Navida.

--------------------------------------------------------------------------------
Writer (to User):

Navida: Navigating the Digital Seas of Innovation

In an era where the digital climate is as fickle and formidable as the ocean's tides, a new visionary emerges on the horizon: Navida. This modern-day titan of technology harnesses the power of innovation to guide businesses and individuals through the ever-changing digital waters.

**The Beacon of Technological Advancement**

Much like a lighthouse guiding ships to safe harbor, Navida stands tall as a beacon of advancement, illuminating the path forward. Its offerings are diverse, ranging from groundbreaking artificial intelligence platforms to robust cybersecurity solutions that shield against the pirates of the digital age. The commitment to seamless user experiences ensures that with Navida, you're not just keeping up, but sailing ahead of the tech curve.

**Chart